In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import io
import math
from functools import reduce
import sys
from copy import deepcopy

def output(a):
    sys.stdout.write(str(a))

N = 9

## Task 1 : Parse the Sudoku data file (5 pts)

In [0]:
data = "/content/drive/My Drive/courses/AI/A3/evil.txt"
def getPuzzle(filename):
    with open(filename, 'r') as f:
        lines = f.read().split('\n')
    board = []
    for line in lines:
        if len(line) != 0:
            line= [int(x) if x.isdigit() else x for z in line for x in str(z)]
            board.append(line)
    board = board[1:][:]
    print(board)
    return board
board = getPuzzle(data)

[[5, 0, 0, 0, 0, 0, 0, 0, 0], [8, 6, 1, 2, 0, 3, 0, 0, 0], [0, 2, 0, 0, 5, 0, 0, 0, 0], [6, 0, 9, 8, 0, 0, 0, 0, 0], [0, 7, 0, 4, 0, 5, 0, 8, 0], [0, 0, 0, 0, 0, 6, 1, 0, 5], [0, 0, 0, 0, 7, 0, 0, 3, 0], [0, 0, 0, 6, 0, 1, 5, 4, 2], [0, 0, 0, 0, 0, 0, 0, 0, 8]]


## Task 2 : Naïve Backtracking Algorithm (30 pts)¶
Implement a naïve backtracking algorithm. The selection of variables and assignment of values can be done either in order or randomly.

In [0]:
def print_board(board):
    # print the sudoku board. 
    if not board:
        output("No solution")
        return
    for i in range(N):
        for j in range(N):
            cell = board[i][j]
            if cell == 0 or isinstance(cell, set):
                output('.')
            else:
                output(cell)
            if (j + 1) % 3 == 0 and j < 8:
                output(' |')

            if j != 8:
                output(' ')
        output('\n')
        if (i + 1) % 3 == 0 and i < 8:
            output("- - - + - - - + - - -\n")


def find_empty(bo):
    # find empty spaces in the board
    for i in range(len(bo)):
        for j in range(len(bo[0])):
            if bo[i][j] == 0:
                return (i, j)  # row, col

    return None

def solve(bo):
    # solve the sudoku by checking for the valid moves
    find = find_empty(bo)
    if not find:
        return True
    else:
        row, col = find

    for i in range(1,10):
        if valid(bo, i, (row, col)):
            bo[row][col] = i

            if solve(bo):
                return True

            bo[row][col] = 0

    return False

def valid(bo, num, pos):
    # Checks for the valid moves.

    # Check row
    for i in range(len(bo[0])):
        if bo[pos[0]][i] == num and pos[1] != i:
            return False

    # Check column
    for i in range(len(bo)):
        if bo[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if bo[i][j] == num and (i,j) != pos:
                return False

    return True
  
print_board(board)


5 . . | . . . | . . .
8 6 1 | 2 . 3 | . . .
. 2 . | . 5 . | . . .
- - - + - - - + - - -
6 . 9 | 8 . . | . . .
. 7 . | 4 . 5 | . 8 .
. . . | . . 6 | 1 . 5
- - - + - - - + - - -
. . . | . 7 . | . 3 .
. . . | 6 . 1 | 5 4 2
. . . | . . . | . . 8


In [0]:
%%time
solve(board)
print_board(board)

5 9 4 | 7 6 8 | 2 1 3
8 6 1 | 2 9 3 | 4 5 7
7 2 3 | 1 5 4 | 8 9 6
- - - + - - - + - - -
6 5 9 | 8 1 7 | 3 2 4
1 7 2 | 4 3 5 | 6 8 9
3 4 8 | 9 2 6 | 1 7 5
- - - + - - - + - - -
4 8 6 | 5 7 2 | 9 3 1
9 3 7 | 6 8 1 | 5 4 2
2 1 5 | 3 4 9 | 7 6 8
CPU times: user 199 ms, sys: 3.18 ms, total: 202 ms
Wall time: 202 ms


## Task 3 : Smart Backtracking Algorithm (40 pts)¶
Incorporate at least one strategy of minimum remaining values (MRV), least constraining value (LCV), and forward checking in your backtracking algorithm.

In [0]:
data = "/content/drive/My Drive/courses/AI/A3/evil.txt"
def getPuzzle(filename):
    # Read the board again.
    with open(filename, 'r') as f:
        lines = f.read().split('\n')
    board = []
    for line in lines:
        if len(line) != 0:
            line= [int(x) if x.isdigit() else x for z in line for x in str(z)]
            board.append(line)
    board = board[1:][:]
    print(board)
    return board
board = getPuzzle(data)

[[5, 0, 0, 0, 0, 0, 0, 0, 0], [8, 6, 1, 2, 0, 3, 0, 0, 0], [0, 2, 0, 0, 5, 0, 0, 0, 0], [6, 0, 9, 8, 0, 0, 0, 0, 0], [0, 7, 0, 4, 0, 5, 0, 8, 0], [0, 0, 0, 0, 0, 6, 1, 0, 5], [0, 0, 0, 0, 7, 0, 0, 3, 0], [0, 0, 0, 6, 0, 1, 5, 4, 2], [0, 0, 0, 0, 0, 0, 0, 0, 8]]


## Using **Forward Checking** in the backtracking algorithm.

In [0]:
def print_board(board):
    # print the sudoku board. 
    if not board:
        output("No solution")
        return
    for i in range(N):
        for j in range(N):
            cell = board[i][j]
            if cell == 0 or isinstance(cell, set):
                output('.')
            else:
                output(cell)
            if (j + 1) % 3 == 0 and j < 8:
                output(' |')

            if j != 8:
                output(' ')
        output('\n')
        if (i + 1) % 3 == 0 and i < 8:
            output("- - - + - - - + - - -\n")

def read(board):
    # Read board into state (replace 0 with set of possible values)

    state = deepcopy(board)
    for i in range(N):
        for j in range(N):
            cell = state[i][j]
            if cell == 0:
                state[i][j] = set(range(1,10))

    return state

state = read(board)


def done(state):
    # Are we done? 

    for row in state:
        for cell in row:
            if isinstance(cell, set):
                return False
    return True


def propagate_step(state):
    # Propagate one step

    new_units = False

    for i in range(N):
        row = state[i]
        values = set([x for x in row if not isinstance(x, set)])
        for j in range(N):
            if isinstance(state[i][j], set):
                state[i][j] -= values
                if len(state[i][j]) == 1:
                    state[i][j] = state[i][j].pop()
                    new_units = True
                elif len(state[i][j]) == 0:
                    return False, None

    for j in range(N):
        column = [state[x][j] for x in range(N)]
        values = set([x for x in column if not isinstance(x, set)])
        for i in range(N):
            if isinstance(state[i][j], set):
                state[i][j] -= values
                if len(state[i][j]) == 1:
                    state[i][j] = state[i][j].pop()
                    new_units = True
                elif len(state[i][j]) == 0:
                    return False, None

    for x in range(3):
        for y in range(3):
            values = set()
            for i in range(3*x, 3*x+3):
                for j in range(3*y, 3*y+3):
                    cell = state[i][j]
                    if not isinstance(cell, set):
                        values.add(cell)
            for i in range(3*x, 3*x+3):
                for j in range(3*y, 3*y+3):
                    if isinstance(state[i][j], set):
                        state[i][j] -= values
                        if len(state[i][j]) == 1:
                            state[i][j] = state[i][j].pop()
                            new_units = True
                        elif len(state[i][j]) == 0:
                            return False, None

    return True, new_units

def propagate(state):
    # Propagate until we reach a fixpoint
    while True:
        solvable, new_unit = propagate_step(state)
        if not solvable:
            return False
        if not new_unit:
            return True


def solve(state):
    # Solve sudoku 

    solvable = propagate(state)

    if not solvable:
        return None

    if done(state):
        return state

    for i in range(N):
        for j in range(N):
            cell = state[i][j]
            if isinstance(cell, set):
                for value in cell:
                    new_state = deepcopy(state)
                    new_state[i][j] = value
                    solved = solve(new_state)
                    if solved is not None:
                        return solved
                return None

print_board(board)

5 . . | . . . | . . .
8 6 1 | 2 . 3 | . . .
. 2 . | . 5 . | . . .
- - - + - - - + - - -
6 . 9 | 8 . . | . . .
. 7 . | 4 . 5 | . 8 .
. . . | . . 6 | 1 . 5
- - - + - - - + - - -
. . . | . 7 . | . 3 .
. . . | 6 . 1 | 5 4 2
. . . | . . . | . . 8


In [0]:
%%time
print_board(solve(state))

5 3 4 | 7 6 8 | 2 9 1
8 6 1 | 2 9 3 | 4 5 7
9 2 7 | 1 5 4 | 8 6 3
- - - + - - - + - - -
6 5 9 | 8 3 7 | 7 2 4
3 7 2 | 4 1 5 | 6 8 6
3 4 8 | 9 2 6 | 1 2 5
- - - + - - - + - - -
4 8 6 | 5 7 2 | 9 3 9
7 9 3 | 6 8 1 | 5 4 2
2 1 5 | 3 4 9 | 6 7 8
CPU times: user 37.1 ms, sys: 3.7 ms, total: 40.8 ms
Wall time: 41 ms



## Task 4 : Analysis and Report (25 pts)
-  Task 1 : Naive backtracking
  - Implemented Naive backtracking and was able to solve every possible puzzle level.
-  Task 2 : Smart backtracking 
  - Implemented Forward Checking and was able to solve every possible level.


|  Levels | Back Tracking Performance |Forward Checking Performance |
| --------| ----------- |-------|
| Given | 13.8 ms | 11.6 ms
| Easy | 18 ms |  6.35 ms |
| Medium | 121 ms | 18.4 ms |
| Hard | 18.3 ms | 9.41 ms |
| Evil | 202 ms | 40.8 ms |

**Conclusion:**
> Based on the above result we can say that Forward checking outperform the Naive backtracking 